Neural Networks and Deep Learning - 3rd Project

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import seaborn as sns
import scipy.io as sio
import time
from keras import  layers, models, Model
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from scipy.stats import skew, kurtosis, entropy
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, UpSampling2D, Concatenate

(x1, y1), (x2, y2) = cifar10.load_data()

Preprocessing the Data

In [2]:
Ntrain = 5000  # Number of training samples
Ntest = 10000   # Number of testing samples

x1, y1 = x1[:Ntrain], y1[:Ntrain]
x2, y2 = x2[:Ntest], y2[:Ntest]

# Pixel normalization
x1norm ,x2norm = x1/255.0, x2/255.0

# Skewness, Kurtosis, Entropy of each channel and Bispectrum of grayscale images
# !! Move the files 'train_skew_kurt_entr.npy' & 'test_skew_kurt_entr.npy' to the same folder with this code file !!
train_stats = np.load('train_skew_kurt_entr.npy')[:Ntrain]
test_stats = np.load('test_skew_kurt_entr.npy')[:Ntest]
# !! Replace Path with the Paths that bispectrum excel files are !!
bispectrum_train = pd.read_excel('C:/Users/samag/OneDrive/Υπολογιστής/9o Εξάμηνο/Neural Networks - Deep Learning/Exercises/ex1/Matlab/train_bispectrum_stats_new.xlsx')[:Ntrain]
bispectrum_test = pd.read_excel('C:/Users/samag/OneDrive/Υπολογιστής/9o Εξάμηνο/Neural Networks - Deep Learning/Exercises/ex1/Matlab/test_bispectrum_stats_new.xlsx')[:Ntest]
x1statistics = np.concatenate((train_stats, bispectrum_train), axis=1)
x2statistics = np.concatenate((test_stats, bispectrum_test), axis=1)

x1skew, x2skew = x1statistics[:,:3], x2statistics[:,:3]
x1kurt, x2kurt = x1statistics[:,3:6], x2statistics[:,3:6]
x1entr, x2entr = x1statistics[:,6:9], x2statistics[:,6:9]
x1bispec, x2bispec = x1statistics[:,9:12], x2statistics[:,9:12]


Hybrid Autoencoder (with encoder format similar to SAM12 neural network)

In [ ]:
# Hybrid Autoencoder

# Encoder
# Convolutional Part
image_input = Input(shape=(32, 32, 3), name="image_input")
cnn_encoder = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
], name="cnn_encoder")
cnn_encoded = cnn_encoder(image_input)

# MLP Part for Skeweness Features (MLP1)
skew_input = Input(shape=(3,), name="skew_input")
skew_encoder = models.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu')
], name="skew_encoder")
skew_encoded = skew_encoder(skew_input)

# MLP Part for Kurtosis Features (MLP2)
kurt_input = Input(shape=(3,), name="kurt_input")
kurt_encoder = models.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu')
], name="kurt_encoder")
kurt_encoded = kurt_encoder(kurt_input)

# MLP Part for Entropy Features (MLP3)
entr_input = Input(shape=(3,), name="entr_input")
entr_encoder = models.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu')
], name="entr_encoder")
entr_encoded = entr_encoder(entr_input)

# MLP Part for Bispectrum Features (MLP4)
bispec_input = Input(shape=(3,), name="bispec_input")
bispec_encoder = models.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu')
], name="bispec_encoder")
bispec_encoded = bispec_encoder(bispec_input)

# Combine Encoded Features
combined_features = layers.concatenate(
    [cnn_encoded, skew_encoded, kurt_encoded, entr_encoded, bispec_encoded],
    name="combined_features"
)

# Decoder
decoder = models.Sequential([
    layers.Dense(256, activation='relu'),
    layers.Reshape((4, 4, 16)),
    layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), activation='relu', padding='same'),
    layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu', padding='same'),
    layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same'),
    layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')  
], name="decoder")
decoded_output = decoder(combined_features)

# Hybrid Autoencoder Model
hybrid_autoencoder = Model(
    inputs=[image_input, skew_input, kurt_input, entr_input, bispec_input],
    outputs=decoded_output,
    name="hybrid_autoencoder"
)


hybrid_autoencoder.compile(optimizer='adam', loss='mse')


hybrid_ae_st = time.time()
history_hybrid_autoencoder =hybrid_autoencoder.fit(
    [x1norm, x1skew, x1kurt, x1entr, x1bispec],
    x1norm,  
    epochs=50,
    batch_size=128
)
hybrid_ae_et = time.time()
hybrid_ae_time = hybrid_ae_et-hybrid_ae_st
print(f"Hybrid Autoencoder Training Time : {hybrid_ae_time:.2f}")



Convolutional Autoencoder 

In [ ]:
# Convolutional Autoencoder 

image_input_simple = Input(shape=(32, 32, 3), name="image_input_simple")

# Encoder
cnn_encoder_simple = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(256, activation='sigmoid'),
], name="cnn_encoder_simple")
cnn_encoded_simple = cnn_encoder_simple(image_input_simple)

# Decoder
decoder_simple = models.Sequential([
    layers.Dense(256, activation='relu'),
    layers.Reshape((4, 4, 16)),
    layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), activation='relu', padding='same'),
    layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu', padding='same'),
    layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same'),
    layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same'),  
], name="decoder_simple")
decoded_output_simple = decoder_simple(cnn_encoded_simple)


# Convolutional Autoencoder Model
cnn_autoencoder = Model(
    inputs=image_input_simple,
    outputs=decoded_output_simple,
    name="cnn_autoencoder"
)

# Compile the Convolutional Autoencoder with MSE Error (Conv AE MSE)
cnn_autoencoder.compile(optimizer='adam', loss='mse')


cnn_ae_st = time.time()
history_cnn_autoencoder = cnn_autoencoder.fit(
    x1norm,  
    x1norm,  
    epochs=50,
    batch_size=128
)
cnn_ae_et = time.time()
cnn_ae_time = cnn_ae_et-cnn_ae_st
print(f"Convolutional Autoencoder with MSE Training Time : {cnn_ae_time:.2f}")


Reconstruction using PCA

In [ ]:
x1norm_flattened = x1norm.reshape(x1norm.shape[0], -1)  

n = 0.9  
pca = PCA(n)

pca_st = time.time()
x1norm_pca = pca.fit_transform(x1norm_flattened)
pca_et = time.time()
pca_time = pca_et - pca_st
print(f"Response Time using PCA : {pca_time:.2f}")


Testing the Models with Test Images

In [ ]:
hybrid_reconstructed = hybrid_autoencoder.predict(
    [x2norm, x2skew, x2kurt, x2entr, x2bispec]
)

cnn_reconstructed = cnn_autoencoder.predict(x2norm)


x2norm_flattened = x2norm.reshape(x2norm.shape[0], -1)
x2norm_pca = pca.transform(x2norm_flattened)
pca_reconstructed = pca.inverse_transform(x2norm_pca)
pca_reconstructed=pca_reconstructed.reshape(x2norm.shape)

# Plot first 5 images
num_images = 5
fig, axes = plt.subplots(4, num_images)
titles = ["Original", "Hybrid AE", "Conv AE MSE", "PCA"]

for i in range(num_images):
    # Original images
    axes[0, i].imshow(x2norm[i])
    axes[0, i].axis("off")
    axes[0, i].set_title("Original")

    # Hybrid AE reconstructions
    axes[1, i].imshow(hybrid_reconstructed[i])
    axes[1, i].axis("off")
    axes[1, i].set_title("Hybrid AE")

    # Conv AE reconstructions
    axes[2, i].imshow(cnn_reconstructed[i])
    axes[2, i].axis("off")
    axes[2, i].set_title("Conv AE MSE")

    # PCA reconstructions
    axes[3, i].imshow(pca_reconstructed[i])
    axes[3, i].axis("off")
    axes[3, i].set_title("PCA")

plt.tight_layout()
plt.show()

# Plot training errors over epochs
plt.figure(figsize=(10, 5))

# Hybrid AE training loss
plt.plot(history_hybrid_autoencoder.history['loss'], label='Hybrid AE')

# Conv AE training loss
plt.plot(history_cnn_autoencoder.history['loss'], label='Conv AE MSE ')

plt.title("Training Loss Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()



for i in range(Ntest):
    mse_hybrid = [mean_squared_error(x2norm[i].flatten(), hybrid_reconstructed[i].flatten())]
    mse_cnn = [mean_squared_error(x2norm[i].flatten(), cnn_reconstructed[i].flatten())]
    mse_pca = [mean_squared_error(x2norm[i].flatten(), pca_reconstructed[i].flatten())]
    
# Average MSE 
avg_mse_hybrid = np.mean(mse_hybrid)
avg_mse_cnn = np.mean(mse_cnn)
avg_mse_pca = np.mean(mse_pca)

# Average MSE Bar chart
models = ['Hybrid AE', 'CNN AE', 'PCA']
avg_mse_values = [avg_mse_hybrid, avg_mse_cnn, avg_mse_pca]

plt.figure(figsize=(8, 6))
plt.bar(models, avg_mse_values, color=['skyblue', 'lightgreen', 'lightcoral'])
plt.title('Average MSE of Reconstructed Images for Test Set')
plt.ylabel('Mean Squared Error')
plt.xlabel('Recontruction Models')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

print(f"Avg MSE of Hybrid AE: {avg_mse_hybrid: .4f}")
print(f"Avg MSE of Conv AE: {avg_mse_cnn: .4f}")
print(f"Avg MSE of Reconstruction using PCA: {avg_mse_pca: .4f}")




Convolutional Autoencoder Using SSIM Loss (Conv AE SSIM)

In [ ]:
# SSIM loss function 0 < SSIM Loss < 2 
def ssim_loss(y_true, y_pred):
    ssim_value = tf.image.ssim(y_true, y_pred, max_val=1.0)  
    return 1 - tf.reduce_mean(ssim_value) 


# Convolutional Autoencoder with SSIM Loss
cnn_autoencoder_ssim = Model(
    inputs=image_input_simple,
    outputs=decoded_output_simple,
    name="cnn_autoencoder_ssim"
)


# Compile the Convolutional Autoencoder with SSIM Loss
cnn_autoencoder_ssim.compile(optimizer='adam', loss=ssim_loss)


cnn_ae_ssim_st = time.time()
history_cnn_autoencoder_ssim = cnn_autoencoder_ssim.fit(
    x1norm,  
    x1norm,  
    epochs=50,
    batch_size=128
)
cnn_ae_ssim_et = time.time()
cnn_ae_ssim_time = cnn_ae_ssim_et - cnn_ae_ssim_st
print(f"Convolutional Autoencoder SSIM Response Time : {cnn_ae_ssim_time:.2f}")


Testing Conv AE SSIM with Test Images

In [ ]:
# Reconstruct images using convolutional autoencoder with SSIM Loss
cnn_reconstructed_ssim = cnn_autoencoder_ssim.predict(x2norm)

num_images = 5      
fig, axes = plt.subplots(3, num_images)
titles = ["Original", "Conv AE MSE", "Conv AE SSIM"]

for i in range(num_images):
    
    axes[0, i].imshow(x2norm[i])
    axes[0, i].axis("off")
    axes[0, i].set_title("Original")

   
    axes[1, i].imshow(cnn_reconstructed[i])
    axes[1, i].axis("off")
    axes[1, i].set_title("Conv AE MSE")

   
    axes[2, i].imshow(cnn_reconstructed_ssim[i])
    axes[2, i].axis("off")
    axes[2, i].set_title("Conv AE SSIM")

plt.tight_layout()
plt.show()


plt.figure(figsize=(10, 5))
plt.plot(history_cnn_autoencoder_ssim.history['loss'], label='Conv AE SSIM')
plt.title("Training Loss Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("SSIM Loss")
plt.legend()
plt.grid()
plt.show()


for i in range(Ntest):
    mse_cnn = [mean_squared_error(x2norm[i].flatten(), cnn_reconstructed[i].flatten())]
    mse_cnn_ssim = [mean_squared_error(x2norm[i].flatten(), cnn_reconstructed_ssim[i].flatten())]



avg_mse_cnn = np.mean(mse_cnn)
avg_mse_cnn_ssim = np.mean(mse_cnn_ssim)

# Average MSE Bar chart
models = ['Conv AE MSE', 'Conv AE SSIM']
avg_mse_values = [avg_mse_cnn, avg_mse_cnn_ssim]

plt.figure(figsize=(8, 6))
plt.bar(models, avg_mse_values, color=['skyblue', 'lightgreen', 'lightcoral'])
plt.title('Average MSE of Reconstructed Images for Test Set')
plt.ylabel('Mean Squared Error')
plt.xlabel('Recontruction Models')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

print(f"Avg MSE of Conv AE: {avg_mse_cnn: .4f}")
print(f"Avg MSE of Conv AE SSIM: {avg_mse_cnn_ssim: .4f}")

